In [1]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import utils

In [2]:
rows = ['Zonal_Winds', 'Meridional_Winds', 'Humidity', 'Air_Temp', 'Sea_Surface_Temp']
train_data = pd.read_csv('../../Cleaning-the-data/tao-cleaned.csv')[rows]
train_label = train_data['Sea_Surface_Temp']

train_data = (train_data - train_data.mean())/train_data.std()
train_data = train_data.values[:15000]
train_label = train_label.values[:15000]
train_data.shape

(15000, 5)

In [3]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence,self).__init__()
        self.lstm1 = nn.LSTM(5,32)
        self.lstm2 = nn.LSTM(32,32)
        self.lstm3 = nn.LSTM(32,1)
 
    def forward(self, seq):
        lstm1_out, _ = self.lstm1(seq, None)
        lstm2_out, _ = self.lstm2(lstm1_out, None)
        lstm3_out, _ = self.lstm3(lstm2_out, None)
        return lstm3_out.squeeze(1)

In [4]:
seq = Sequence()
test_data = torch.Tensor(train_data[:10]).view(-1,1,5)
test_label = torch.Tensor(train_data[:10][:,-1]).view(-1,1)

print(test_label)
print(seq(test_data))

tensor([[-0.7540],
        [-0.8920],
        [-1.2446],
        [-1.7403],
        [-2.3127],
        [-2.0827],
        [-1.7250],
        [-1.8272],
        [-1.7761],
        [-0.9073]])
tensor([[0.1106],
        [0.1400],
        [0.1473],
        [0.1495],
        [0.1508],
        [0.1515],
        [0.1521],
        [0.1529],
        [0.1539],
        [0.1564]], grad_fn=<SqueezeBackward1>)


In [5]:
criterion = nn.MSELoss()
optimizer = optim.Adam(seq.parameters(), lr=0.01)
seq_size = 50

In [6]:
for epoch in range(50):
    running_loss = 0
    running_error = 0
    num_batches = 0
    for i in range(0, 15000, seq_size):
        data = train_data[i:i+seq_size]
        xs = torch.Tensor(data[:-1]).view(-1, 1, 5)
        ys = torch.Tensor(data[1:][:,-1]).view(-1,1)
        
        lstm_out = seq(xs)
        loss = criterion(lstm_out.view(1, 49), ys.view(1, 49))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # compute some stats:
        
        num_batches+=1
        
        with torch.no_grad():
            running_loss += loss.item()
            error = utils.get_error(lstm_out.view(1,-1) , ys)
            running_error += error 

    # once the epoch is finished we divide the "running quantities"
    # by the number of batches
    if epoch % 1 == 0:
        total_loss = running_loss/num_batches
        total_error = running_error/num_batches

        print('epoch=',epoch, '\t loss=', total_loss, '\t error={:2.2%}'.format(total_error) ,'percent')

epoch= 0 	 loss= 1.2320923470209042 	 error=99.01% percent
epoch= 1 	 loss= 1.0633734791361107 	 error=98.59% percent
epoch= 2 	 loss= 1.0320508504783115 	 error=97.01% percent
epoch= 3 	 loss= 1.0269222522018147 	 error=95.90% percent
epoch= 4 	 loss= 1.0243883091351502 	 error=96.01% percent
epoch= 5 	 loss= 1.0228305982978296 	 error=96.46% percent
epoch= 6 	 loss= 1.0219315989052604 	 error=96.45% percent
epoch= 7 	 loss= 1.0212241234219983 	 error=96.40% percent
epoch= 8 	 loss= 1.0210797725752248 	 error=96.45% percent
epoch= 9 	 loss= 1.0217733706696164 	 error=96.91% percent
epoch= 10 	 loss= 1.0216427395721743 	 error=96.76% percent
epoch= 11 	 loss= 1.0201457674452832 	 error=96.24% percent
epoch= 12 	 loss= 1.0241159292745095 	 error=97.16% percent
epoch= 13 	 loss= 1.0270926825216156 	 error=97.31% percent
epoch= 14 	 loss= 1.021453098618076 	 error=95.73% percent
epoch= 15 	 loss= 1.020008927584519 	 error=94.79% percent
epoch= 16 	 loss= 1.019508821488198 	 error=95.06% p

KeyboardInterrupt: 